<a href="https://colab.research.google.com/github/Ha-eunYu/OTTO/blob/main/otto_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install polars